In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
tables = ['register', 'users']
for table in tables:
    display_table(table)

+------------+---------+
| contest_id | user_id |
+------------+---------+
|    207     |    2    |
|    208     |    2    |
|    208     |    6    |
|    208     |    7    |
|    209     |    2    |
|    209     |    6    |
|    209     |    7    |
|    210     |    2    |
|    210     |    6    |
|    210     |    7    |
|    215     |    6    |
|    215     |    7    |
+------------+---------+
+---------+-----------+
| user_id | user_name |
+---------+-----------+
|    2    |    Bob    |
|    6    |   Alice   |
|    7    |    Alex   |
+---------+-----------+


### Write an SQL query to find the percentage of the users registered in each contest rounded to two decimals.Return the result table ordered by percentage in descending order. In case of a tie, order it by  `contest_id` in `ascending` order.

```
+------------+------------+
| contest_id | percentage |
+------------+------------+
| 208        | 100.0      |
| 209        | 100.0      |
| 210        | 100.0      |
| 215        | 66.67      |
| 207        | 33.33      |
+------------+------------+
Explanation: 
All the users registered in contests 208, 209, and 210. The percentage is 100% and we sort them in the answer table by contest_id in ascending order.
Alice and Alex registered in contest 215 and the percentage is ((2/3) * 100) = 66.67%
Bob registered in contest 207 and the percentage is ((1/3) * 100) = 33.33%
```

In [3]:
%%sql 

SELECT contest_id, COUNT(contest_id) AS contest_count 
FROM users u 
JOIN register r ON r.user_id = u.user_id
GROUP BY contest_id

contest_id,contest_count
207,1
208,3
209,3
210,3
215,2


In [4]:
%%sql 

SELECT contest_id, (SELECT COUNT(user_id) FROM users) AS user_count
FROM users u 
JOIN register r ON r.user_id = u.user_id
GROUP BY contest_id

contest_id,user_count
207,3
208,3
209,3
210,3
215,3


In [5]:
%%sql 

SELECT contest_id, ROUND((COUNT(contest_id) / (SELECT COUNT(user_id) FROM users)) * 100, 2) AS percentage
FROM users u 
JOIN register r ON r.user_id = u.user_id
GROUP BY contest_id
ORDER BY percentage DESC, contest_id


contest_id,percentage
208,100.00
209,100.00
210,100.00
215,66.67
207,33.33


# Using Pandas

In [6]:
import pandas as pd 

In [7]:
register_query = %sql SELECT * FROM register # type: ignore 
users_query = %sql SELECT * FROM users # type: ignore 

register_df = register_query.DataFrame()
users_df = users_query.DataFrame()

display(register_df, users_df)

,contest_id,user_id
0,207,2
1,208,2
2,208,6
3,208,7
4,209,2
5,209,6
6,209,7
7,210,2
8,210,6
9,210,7


,user_id,user_name
0,2,Bob
1,6,Alice
2,7,Alex


In [8]:
merged_df = users_df.merge(register_df, on='user_id', how='outer')
merged_df

,user_id,user_name,contest_id
0,2,Bob,207
1,2,Bob,208
2,2,Bob,209
3,2,Bob,210
4,6,Alice,208
5,6,Alice,209
6,6,Alice,210
7,6,Alice,215
8,7,Alex,208
9,7,Alex,209


In [9]:
users_df.value_counts()

user_id  user_name
2        Bob          1
6        Alice        1
7        Alex         1
Name: count, dtype: int64

In [10]:
users_df.value_counts().sum()

3

In [11]:
register_df.groupby('contest_id').agg({'user_id': 'count'})

,user_id
contest_id,
207,1
208,3
209,3
210,3
215,2


In [12]:
register_df. \
    groupby('contest_id') \
    .agg({'user_id': 'count'}) / users_df.value_counts().sum()

,user_id
contest_id,
207,0.333333
208,1.000000
209,1.000000
210,1.000000
215,0.666667


In [13]:
result_df = register_df. \
    groupby('contest_id') \
    .agg({'user_id': 'count'}) / users_df.value_counts() \
    .sum()

result_df.round(2)

,user_id
contest_id,
207,0.33
208,1.00
209,1.00
210,1.00
215,0.67


In [14]:
result_df = result_df.round(2)
result_df.rename(columns={'user_id': 'percentage'})

,percentage
contest_id,
207,0.33
208,1.00
209,1.00
210,1.00
215,0.67


In [15]:
result_df = result_df.rename(columns={'user_id': 'percentage'}) * 100
result_df

,percentage
contest_id,
207,33.0
208,100.0
209,100.0
210,100.0
215,67.0


In [16]:
result_df.sort_values(by='percentage', ascending=False)

,percentage
contest_id,
208,100.0
209,100.0
210,100.0
215,67.0
207,33.0


In [17]:
# perform a secondary sort
result_df.sort_values(by=['percentage', 'contest_id'], ascending=[False, True])


,percentage
contest_id,
208,100.0
209,100.0
210,100.0
215,67.0
207,33.0
